<a href="https://colab.research.google.com/github/nmarkin/DL_corse/blob/main/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [6]:
def load_planar_dataset():
    np.random.seed(1)
    m = 400 # number of examples
    N = int(m/2) # number of points per class
    D = 2 # dimensionality
    X = np.zeros((m,D)) # data matrix where each row is a single example
    Y = np.zeros((m,1), dtype='uint8') # labels vector (0 for red, 1 for blue)
    a = 4 # maximum ray of the flower

    for j in range(2):
        ix = range(N*j,N*(j+1))
        t = np.linspace(j*3.12,(j+1)*3.12,N) + np.random.randn(N)*0.2 # theta
        r = a*np.sin(4*t) + np.random.randn(N)*0.2 # radius
        X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
        Y[ix] = j
        
    X = X.T
    Y = Y.T

    return X, Y

In [7]:
class my_dataset(Dataset):
    def __init__(self, X, Y):
        self.x = torch.tensor(X)
        self.y = torch.tensor(Y)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.y.shape[0]


In [47]:
X, Y = load_planar_dataset()
X_train, X_test, y_train, y_test = train_test_split(X.T, Y.T, random_state=42)
y_train = y_train.reshape(len(y_train))
y_test = y_test.reshape(len(y_test))
# print(y_train)
train, test = my_dataset(X_train, y_train), my_dataset(X_test, y_test) 
train_loader = DataLoader(dataset=train, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test, shuffle=True)

In [48]:
for x,y in train_loader:
  print(x.shape, y.shape)
  break

torch.Size([64, 2]) torch.Size([64])


In [103]:
model = nn.Sequential(
    nn.Linear(2, 1024),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.2),
    nn.ReLU(),

    nn.Linear(1024, 1024),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.2),
    nn.ReLU(),

    nn.Linear(1024, 1024),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.2),
    nn.ReLU(),

    nn.Linear(1024, 2),
    nn.Softmax()
)

In [118]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-10)

In [112]:
model.cuda()

Sequential(
  (0): Linear(in_features=2, out_features=1024, bias=True)
  (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Dropout(p=0.2, inplace=False)
  (3): ReLU()
  (4): Linear(in_features=1024, out_features=1024, bias=True)
  (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Dropout(p=0.2, inplace=False)
  (7): ReLU()
  (8): Linear(in_features=1024, out_features=1024, bias=True)
  (9): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): Dropout(p=0.2, inplace=False)
  (11): ReLU()
  (12): Linear(in_features=1024, out_features=2, bias=True)
  (13): Softmax(dim=None)
)

In [119]:
epochs = 200
for i in range(epochs):
  for j, (x, y) in enumerate(train_loader):
    optimizer.zero_grad() # to get rid of previous gradients
    x = x.cuda()
    y = y.cuda()
    y_pred = model(x.float())
    # y_pred = torch.tensor([0 if x[0] > x[1] else 1 for x in y_pred]).view(64).cuda()
    loss = criterion(y_pred, y.long())
    print(f"Epoch {i}\t iter {j}\t loss {loss}")
    loss.backward() # calculates all gradients
    optimizer.step() 

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch 0	 iter 0	 loss 0.6102481484413147
Epoch 0	 iter 1	 loss 0.5788864493370056
Epoch 0	 iter 2	 loss 0.563368558883667
Epoch 0	 iter 3	 loss 0.5164055824279785
Epoch 0	 iter 4	 loss 0.5178093314170837
Epoch 1	 iter 0	 loss 0.5947136878967285
Epoch 1	 iter 1	 loss 0.5320119261741638
Epoch 1	 iter 2	 loss 0.578904390335083
Epoch 1	 iter 3	 loss 0.5476478934288025
Epoch 1	 iter 4	 loss 0.5405449867248535
Epoch 2	 iter 0	 loss 0.5633649230003357
Epoch 2	 iter 1	 loss 0.532014787197113
Epoch 2	 iter 2	 loss 0.5790011882781982
Epoch 2	 iter 3	 loss 0.5789035558700562
Epoch 2	 iter 4	 loss 0.5405355095863342
Epoch 3	 iter 0	 loss 0.5476368069648743
Epoch 3	 iter 1	 loss 0.5633686780929565
Epoch 3	 iter 2	 loss 0.4695356488227844
Epoch 3	 iter 3	 loss 0.6413863897323608
Epoch 3	 iter 4	 loss 0.5861453413963318
Epoch 4	 iter 0	 loss 0.5789070725440979
Epoch 4	 iter 1	 loss 0.5164015293121338
Epoch 4	 iter 2	 loss 0.6258620023727417
Epoch 4	 iter 3	 loss 0.5320117473602295
Epoch 4	 iter 4	 lo

In [121]:
model.eval()
preds = []
true = []

for (x,y) in test_loader:
    x = x.cuda()
    y = y.cuda()
    # print(model(x.float()).data)
    # print(torch.max(model(x.float()).data, 1))
    preds.append(int(torch.max(model(x.float()).data, 1)[1]))
    true.append(int(y))

print(accuracy_score(true, preds))

0.75


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
